<h1> 0. Scrapping de la liste des entreprises </h1> 
<h3> Structure du site </h3>
page principale
    catégories
        liste des entreprises notées
            page des entreprises avec impression des avis

<h3> Stratégie de scrappping </h3>

1) Choisir une catégorie 
2) Scrap 1 : depuis la page des listes de entreprises:
    scrapper:
- nom de l'entreprise
- trust score
- nombre d'avis
- lien vers la page de l'entreprise

<u> -> Sauvegarder comme df/csv "liste_entreprises" </u>

In [2]:
# librairies
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd
import random
import math

<h1> Catégorie de test: Immobilier </h1>
pour tester j'essaie avec la catégorie immobilier qui a de nombreuses entreprises et beaucoup d'avis et un métier qui me semble assez riche.

<h1> 0. Scrapping de la liste des entreprises immobilières </h1>

In [4]:
# Aquisition de la page catégorie immobilier
# structure de la page

"""
page principale immo: https://fr.trustpilot.com/categories/real_estate_agents
il y a 5 pages, je vais préparer une for loop sur 5 pages

Les avis immo sont trop déséquilibrés (4 ou 5 étoiles ++)
page principale banque: https://fr.trustpilot.com/categories/bank

"""
# page de départ
url = "https://fr.trustpilot.com/categories/bank"
# clé de page qui se rajoute à l'URL de départ 
clé_pages = "?page="

In [5]:
# test de process des 5 pages
"""
Probleme a partr de la page 4, il n'y a qu'un score et n avis tandis que les autres variables
ont une longueur de 20
Solution: remplir les echecs de scrap par des NA? à moins qu'il n'y ait qu'une 
balise de ce type sur cette page, auquel cas, voir comment capturer l'ancienne version et merge?
"""
liste_entreprises = pd.DataFrame()

# aquisition de la page
print(url)
immo = requests.get(url)
soup = BeautifulSoup(immo.content, "lxml")

n_entreprises = soup.find("p", class_ = "typography_body-m__xgxZ_ typography_appearance-default__AAY17")
n_entreprises = int(n_entreprises.text.split(sep=" ")[2])
n_pages_ent = n_pages = math.ceil(n_entreprises / 20)

# tourne tant que le décompte et plus petit que le nombre de pages souhaitées
for ent_pages in range(0,n_pages_ent):
    # définition de l'url de la page
    scrap_url = url+clé_pages+str(ent_pages)
    # aquisition de la page
    print(scrap_url)
    html = requests.get(scrap_url)
    soup = BeautifulSoup(html.content, "lxml")
    
    cartes_entreprises = soup.find_all("div", class_ = "paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_wrapper__2JOo2")
    for carte in cartes_entreprises:
        # extraction des noms des entreprises
        nom = []
        nom_html = carte.find("p", class_="typography_heading-xs__jSwUz typography_appearance-default__AAY17 styles_displayName__GOhL2")
        if nom_html is not None:
            nom.append(nom_html.text)
        else:
            nom.append("na")
        # extraction de l'ensemble score et avis
        score,n_avis, test = [],[],[]

        # score bis

        score_html = carte.find("span", class_ = "typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_trustScore__8emxJ")
        if score_html is not None:
            score_html = score_html.text.replace("TrustScore ","")
            score.append(score_html)
        else:
            score.append("na")

        # avis
        avis_html = carte.find("p", class_ = "typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_ratingText__yQ5S7")
        if avis_html is not None:
            avis_html = avis_html.text.split(" ")[1].split("|")[1]
            n_avis.append(avis_html)
        else:
            n_avis.append('na')

        # lien
        url_page_entreprise = []
        url_page_entreprise_html = carte.find("a", attrs={"name":"business-unit-card",
                                                        "target": "_self",
                                                        "class":"link_internal__7XN06 link_wrapper__5ZJEx styles_linkWrapper__UWs5j"})
        if url_page_entreprise_html is not None:
            url_page_entreprise_html = url_page_entreprise_html["href"]
            url_page_entreprise.append(url_page_entreprise_html)
        else:
            url_page_entreprise.append('na')      

        # création de la dataframe
        print("juste avant création dataframe n")
        print("nom",nom,"\n","score", score,"\n", "n_avis",n_avis,"\n")
        liste_entreprise_temp = pd.DataFrame({"nom":nom,
                                "score": score,
                                "n_avis": n_avis,
                                "url_page_entreprise": url_page_entreprise})

        liste_entreprises = pd.concat([liste_entreprises,liste_entreprise_temp], ignore_index= True)
        liste_entreprise_temp = pd.DataFrame()





https://fr.trustpilot.com/categories/bank
https://fr.trustpilot.com/categories/bank?page=0
https://fr.trustpilot.com/categories/bank?page=1
juste avant création dataframe n
nom ['Younited Credit'] 
 score ['4,8'] 
 n_avis ['66\u202f633'] 

juste avant création dataframe n
nom ['CIB Formation'] 
 score ['4,8'] 
 n_avis ['549'] 

juste avant création dataframe n
nom ['Cofidis'] 
 score ['4,7'] 
 n_avis ['23\u202f486'] 

juste avant création dataframe n
nom ['Qonto'] 
 score ['4,7'] 
 n_avis ['13\u202f976'] 

juste avant création dataframe n
nom ['Pixpay'] 
 score ['4,7'] 
 n_avis ['5\u202f874'] 

juste avant création dataframe n
nom ['Shine'] 
 score ['4,6'] 
 n_avis ['3\u202f507'] 

juste avant création dataframe n
nom ['Money Walkie'] 
 score ['4,6'] 
 n_avis ['877'] 

juste avant création dataframe n
nom ['PennyPet'] 
 score ['4,6'] 
 n_avis ['61'] 

juste avant création dataframe n
nom ['Anytime'] 
 score ['4,5'] 
 n_avis ['7\u202f142'] 

juste avant création dataframe n
nom ['Bourso

In [6]:
liste_entreprises.sort_values('n_avis')

,nom,score,n_avis,url_page_entreprise
21,Cofidis - O Cr&eacute;dito Por Telefone,"4,7",115,/review/cofidis.pt
27,MoneyNow,"4,4",12,/review/moneynow.fr
9,BoursoBank,"4,4",13 830,/review/boursobank.com
3,Qonto,"4,7",13 976,/review/qonto.com
23,Sumeria,"4,5",14 026,/review/sumeria.eu
25,Orange Bank,"4,4",14 858,/review/orangebank.fr
10,propulsebyca.fr,"4,4",158,/review/propulsebyca.fr
38,Plusq1taux Landes Pays Basque,"3,9",19,/review/plusq1taux.com
22,Kard,"4,6",1 884,/review/kard.eu
26,Louvre Banque Privée,"4,4",234,/review/bpe.fr


In [7]:
# sauvegarder le df 
liste_entreprises.to_csv("../data/liste_entreprises_banque.csv")